### Networking

- 프로젝트는 가장 바깥쪽에 있는 컨테이너로, 동일한 신뢰 경계를 공유하는 리소스를 그룹화하는 데 사용됩니다. 프로젝트마다 자체 액세스 정책(IAM) 및 구성원 목록이 있으므로 많은 개발자가 프로젝트를 팀에 매핑합니다. 프로젝트는 리소스 사용량을 반영하는 청구 및 할당량 세부정보 수집기의 역할도 합니다. 프로젝트에는 서브네트워크, 방화벽 규칙 및 경로를 포함하는 네트워크가 포함됩니다

- 네트워크는 리소스를 서로 간에 직접 연결하며 외부 네트워크에도 연결합니다. 네트워크는 방화벽을 사용하여 인바운드 및 아웃바운드 연결에 대한 액세스 정책도 관리합니다. 네트워크에는 전역 네트워크(여러 리전에 걸쳐 수평 확장 가능)와 리전 네트워크(단일 리전 내 짧은 지연 시간 제공)가 있습니다.

- 서브네트워크를 사용하면 관련 리소스(Compute Engine 인스턴스)를 RFC1918 비공개 주소 공간으로 그룹화할 수 있습니다. 서브네트워크는 리전별로만 구성할 수 있습니다. 서브네트워크는 자동 모드 또는 커스텀 모드 네트워크에 포함될 수 있습니다.
    - 자동 모드 네트워크에는 리전별로 하나의 서브넷이 있으며 각각 사전 정의된 IP 범위 및 게이트웨이를 포함합니다. 이러한 서브넷은 자동 모드 네트워크를 만들 때 자동으로 만들어지며 각 서브넷은 전체 네트워크와 동일한 이름을 갖습니다.
     - 커스텀 모드 네트워크는 생성 시에 서브넷을 포함하지 않습니다. 커스텀 모드 네트워크에서 인스턴스를 만들려면 먼저 해당 리전에 서브네트워크를 만들고 해당 IP 범위를 지정해야 합니다. 커스텀 모드 네트워크에는 리전마다 서브넷이 하나도 없거나 하나 또는 여러 개가 있을 수 있습니다.

### Region 설정
>gcloud config set compute/zone "us-east1-c"
export ZONE=$(gcloud config get compute/zone)

>gcloud config set compute/region "us-east1"
export REGION=$(gcloud config get compute/region)

### Ingressed firewall rules
|설정|설명|
|---|---|
|default-allow-internal |네트워크상의 인스턴스 간에 모든 프로토콜 및 포트의 네트워크 연결을 허용합니다.|
|default-allow-ssh | 모든 소스에서 TCP 포트 22를 사용하여 네트워크의 모든 인스턴스에 SSH로 연결할 수 있도록 허용합니다.|
|default-allow-rdp | 모든 소스에서 TCP 포트 3389를 사용하여 네트워크의 모든 인스턴스에 RDP로 연결할 수 있도록 허용합니다.|
|default-allow-icmp | 모든 소스에서 네트워크의 모든 인스턴스에 이르는 ICMP 트래픽을 허용합니다.|

- 기본 방화벽 규칙을 검토하려면 Cloud 콘솔에서 탐색 메뉴 > VPC 네트워크 > 방화벽을 클릭합니다.
- 기본 경로를 검토하려면 탐색 메뉴 > VPC 네트워크 > 경로를 클릭하고 > 네트워크 및 리전을 선택하여 경로를 확인합니다.

### 콘솔을 사용하여 커스텀 네트워크 생성

- VNC Network -> Create a VNC Network
    - name 입력
        - Subnet 생성 > name, region, IPv4 range 입력
        - IPv4 입력예) 0.0.0.0/16
        - Done
        - ADD SUBNET으로 서브넷을 추가
    - create로 vnc network 생성

### Cloud Shell을 사용하여 커스텀 네트워크 생성
>gcloud compute networks create taw-custom-network --subnet-mode custom

- 서브넷 생성 예

>gcloud compute networks subnets create subnet-us-east1 \
   --network taw-custom-network \
   --region us-east1 \
   --range 10.0.0.0/16

- 생성한 네트워크 보기

>gcloud compute networks subnets list \
   --network taw-custom-network

### 방화벽 규칙 추가
- 네트워크 및 방화벽에서 인스턴스 태그를 사용하여 태그된 VM 인스턴스에 특정 방화벽 규칙을 적용합니다. 예를 들어 대규모 웹사이트 제공과 같이 동일한 작업을 수행하는 인스턴스가 여러 개 있는 경우 공유된 단어나 용어를 사용하여 이러한 인스턴스에 태그를 지정한 다음 해당 태그를 사용하여 방화벽 규칙으로 해당 인스턴스에 대한 HTTP 액세스를 허용할 수 있습니다.
- 태그는 메타데이터 서버에도 반영되므로 인스턴스에서 실행되는 애플리케이션에 사용할 수 있습니다.

#### 콘솔을 통해 방화벽 규칙 추가
- VPC Network -> 수정하려는 vpc 선택 -> firewalls 탭 선택 -> add firewall rule 선택
- Targets 중 Specified target tags 선택 시 아래 Target tags에 대상 태그 입력 예)http
- Source filter, source ipv4 ranges에서 0.0.0.0/0 입력 시 모든 IP에 대해서 방화벽을 열기
- Protocols and ports에서 특정 포트만 열 경우, TCP, UDP 선택 후 포트번호 입력 예)TCP 80

#### 쉘을 통해 방화벽 규칙 추가
>gcloud compute firewall-rules create nw101-allow-http \
--allow tcp:80 --network taw-custom-network --source-ranges 0.0.0.0/0 \
--target-tags http

### VM 생성 시 영역 지정
>gcloud compute instances create us-test-01 \
--subnet subnet-us-east1 \
--zone us-east1-c \
--machine-type e2-standard-2 \
--tags ssh,http,rules

### Traceroute
- Traceroute는 두 호스트 간 경로를 추적하는 도구입니다. Traceroute는 다양한 유형의 네트워크 문제 파악에 있어 유용한 첫 번째 단계가 될 수 있습니다. 지원팀이나 네트워크 엔지니어가 네트워크 문제를 진단할 때 Traceroute를 요청하는 경우가 많습니다.

- Traceroute에 호스트 간의 모든 계층 3(라우팅 계층) 홉이 표시됩니다. 이는 TTL(수명) 값을 1부터 증가시켜 패킷을 원격 대상에 전송하는 방식으로 이루어집니다. TTL 필드는 라우터마다 하나씩 줄어드는 IP 패킷의 필드입니다. TTL이 0이 되면 패킷은 삭제되고 발송자에게 'TTL 초과' ICMP 메시지가 반환됩니다. 이 방법은 라우팅 루프를 방지하는 데 사용되며 TTL 필드가 결국 0까지 감소하기 때문에 패킷이 무한하게 루프로 돌지 않습니다. 기본적으로 OS는 TTL 값을 높은 값(64, 128, 255 등)으로 설정하므로 루프 현상은 비정상적인 상황에서만 발생합니다.

- Traceroute는 TTL 값이 1인 패킷을 먼저 보낸 다음 2, 3 등의 순서로 보내므로 이러한 패킷은 경로의 첫 번째, 두 번째 등의 라우터에서 만료됩니다. 그런 다음 반환되는 ICMP TTL 초과 메시지의 소스 IP/호스트를 가져와 중간 홉의 이름/IP를 표시합니다. TTL이 충분히 높아지면 패킷이 대상에 도달하고 대상에서 응답합니다.

- 전송되는 패킷 유형은 구현 방식에 따라 다릅니다. Linux에서 UDP 패킷은 사용되지 않는 높은 포트로 전송됩니다. 따라서 최종 대상에서는 'ICMP 포트 연결할 수 없음'으로 응답합니다. Windows 및 MTR 도구는 기본적으로 ICMP 에코 요청(예: 핑)을 사용하므로 최종 대상에서 ICMP 에코 응답으로 응답합니다.